In [ ]:
repo_path="/home/students/s407535/Desktop/mieszkania3"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

%matplotlib inline
def zerojeden(df):
    for j in range(0,len(df)):
        if (df[j,1]==' więcej niż 8'):
            df[j,1]="10"
    df=df.astype(np.float)
    col_mean = np.nanmean(df, axis=0)
    inds = np.where(np.isnan(df))
    df[inds] = np.take(col_mean, inds[1])
    return df

train_data_path = "{}/train/train.tsv".format(repo_path)
train_data = pd.read_csv(train_data_path, delimiter='\t', encoding='utf-8', engine='python')

X_columns = ['Powierzchnia w m2', 'Liczba pokoi']
y_column = ['cena']

X = train_data[X_columns].values
y = train_data[y_column].values

print("X shape:", X.shape)
print("Y shape:", y.shape)

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X,y)

valid_data_X = pd.read_csv("{}/dev-0/in.tsv".format(repo_path),
                           delimiter='\t',
                           encoding='utf-8',
                           engine='python',
                           names=train_data.columns[1:])

valid_data_y = pd.read_csv("{}/dev-0/expected.tsv".format(repo_path),
                           delimiter='\t',
                           encoding='utf-8',
                           engine='python',
                           names=train_data.columns[:1])

X_valid = valid_data_X[X_columns].values
y_valid = valid_data_y[y_column].values

test_data_X = pd.read_csv("{}/test-A/in.tsv".format(repo_path),
                           delimiter='\t',
                           encoding='utf-8',
                           names=train_data.columns[1:],
                           lineterminator='\n')


X_test = test_data_X[X_columns].values
zerojeden(X_test)

valid_predicted = model.predict(X_valid)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_valid, model.predict(X_valid)))
print(format(model.score(X_valid, y_valid)), format(rmse))

test_predicted = model.predict(X_test)

pd.Series(valid_predicted.flatten()).to_csv("{}/dev-0/out.tsv".format(repo_path),
                                            sep='\t',
                                            index=None)

pd.Series(test_predicted.flatten()).to_csv("{}/test-A/out.tsv".format(repo_path),
                                           sep='\t',
                                           index=None)